In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
% matplotlib inline

In [3]:
with open('./data/nomad_seawifs_v2.a2_2008200.txt', 'r') as f:
    for line in f:
        if 'fields=' in line:
            break
columns = line.strip().strip('/fields=').split(',')

In [4]:
df = pd.read_csv('./data/nomad_seawifs_v2.a2_2008200.txt', names=columns, skiprows=107)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Columns: 243 entries, year to va
dtypes: float64(226), int64(15), object(2)
memory usage: 941.7+ KB


In [6]:
df_rc = pd.read_csv('./data/Rayleigh&Fresnel_corrected_Rrc.txt', sep='\t')

In [7]:
df_rc.head()

,filename,lat,lon,Rrs_412,Rrs_443,Rrs_490,Rrs_510,Rrs_555,Rrs_670
0,S1997284110316.L2_MLAC.hdf,39.2900,25.1100,0.012088,0.012417,0.011739,0.010579,0.009110,0.006655
1,S2000053153433.L2_MLAC.hdf,-61.4500,-62.2990,0.010525,0.010636,0.009614,0.007913,0.006224,0.004794
2,S2001050135427.L2_MLAC.hdf,-61.2900,-56.2900,0.004443,0.004387,0.004240,0.003686,0.002646,0.001177
3,S2002022133012.L2_MLAC.hdf,-60.9990,-56.4980,0.005869,0.005866,0.005535,0.004643,0.003326,0.001747
4,S1997270134451.L2_MLAC.hdf,24.1392,-20.9995,0.009464,0.008968,0.007719,0.005974,0.004161,0.002517


In [8]:
df_rc['filename'] = df_rc.filename.str.strip('.hdf')

In [9]:
# checking to see if both datasets are congruent

sat_files = df.sat_file.tolist()
lat1 = df.lat.tolist()
lon1 = df.lon.tolist()

filenames = df_rc.filename.tolist()
lat2 = df_rc.lat.tolist()
lon2 = df_rc.lon.tolist()

for i, (s, f, lt1, lt2, ln1, ln2) in enumerate (zip(sat_files, filenames, lat1,
                                                    lat2, lon1, lon2)):
    if s != f:
        print(f'fname#{i}: {s}<->{f}')
    if lt1 != lt2:
        print(f'lat#{i}: {lt1}<->{lt2}')
    if ln1 != ln2:
        print(f'lon#{i}: {ln1}<->{ln2}')

In [10]:
def convert_to_dt(row):
    dt_str = f'{row.year}-{row.month}-{row.day} {row.hour}:{row.minute}:{row.second}'
    return pd.to_datetime(dt_str, format='%Y-%m-%d %H:%M:%S')

# consolidate date and time columns into single datetime type column
df.insert(0, 'datetime', df.apply(convert_to_dt, axis=1))
df.drop(['year', 'month', 'day', 'hour', 'minute', 'second'], axis=1, inplace=True)

In [11]:
df_rc.rename(columns=dict(Rrs_412='sat_rho_rc412', Rrs_443='sat_rho_rc443',
                          Rrs_490='sat_rho_rc490', Rrs_510='sat_rho_rc510',
                          Rrs_555='sat_rho_rc555', Rrs_670='sat_rho_rc670'),
           inplace=True)

In [12]:
df_2 = pd.merge(df, df_rc, left_index=True, right_index=True)
df_2.rename(columns={'lat_x': 'lat', 'lon_x': 'lon'}, inplace=True)
df_2.drop(['lat_y', 'lon_y'], axis=1, inplace=True)

In [13]:
df.to_pickle('./pickleJar/df_0_NMD_SWF_v2_a2_2008200.pkl')
df_rc.to_pickle('./pickleJar/df_0_R&F_Corr.pkl')
df_2.to_pickle('./pickleJar/df_1_merged.pkl')

#### <u>Cleaning data</u>

In [14]:
df_2.describe().T

,count,mean,std,min,25%,50%,75%,max
lat,495.0,28.887491,20.588358,-67.643000,26.936900,32.400000,42.350000,79.000000
lon,495.0,-70.959878,41.512573,-170.198000,-83.391000,-76.013200,-66.999000,171.716000
id,495.0,4439.327273,2405.666837,92.000000,2036.500000,4192.000000,6666.500000,7831.000000
oisst,495.0,19.066020,6.983298,0.780000,13.775000,19.600000,25.120000,30.510000
etopo2,495.0,768.961616,1410.753061,0.000000,20.500000,156.000000,519.000000,5529.000000
chl,495.0,-263.342879,441.781960,-999.000000,-999.000000,0.246400,0.903000,43.100000
chl_a,495.0,-679.385258,467.258748,-999.000000,-999.000000,-999.000000,0.178670,23.984000
kd405,495.0,-999.000000,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000
kd411,495.0,-657.840220,474.310189,-999.000000,-999.000000,-999.000000,0.055900,2.887300
kd443,495.0,-647.760552,477.551639,-999.000000,-999.000000,-999.000000,0.052160,2.476800


In [19]:
df_2.replace(-999, np.NaN, inplace=True)

In [20]:
df_2.describe().T

,count,mean,std,min,25%,50%,75%,max
lat,495.0,28.887491,20.588358,-67.643000,26.936900,32.400000,42.350000,79.000000
lon,495.0,-70.959878,41.512573,-170.198000,-83.391000,-76.013200,-66.999000,171.716000
id,495.0,4439.327273,2405.666837,92.000000,2036.500000,4192.000000,6666.500000,7831.000000
oisst,495.0,19.066020,6.983298,0.780000,13.775000,19.600000,25.120000,30.510000
etopo2,495.0,768.961616,1410.753061,0.000000,20.500000,156.000000,519.000000,5529.000000
chl,364.0,1.412843,3.215504,0.025000,0.205895,0.514400,1.235225,43.100000
chl_a,158.0,2.324667,4.019108,0.027000,0.221500,0.815785,2.170500,23.984000
kd405,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kd411,169.0,0.254977,0.413295,0.021060,0.052140,0.115470,0.217120,2.887300
kd443,174.0,0.215671,0.332184,0.019000,0.046753,0.102180,0.217072,2.476800


In [17]:
df_2.filter(regex='sat').loc[df_2.sat_lt412==0]

,sat_rrs412,sat_rrs443,sat_rrs490,sat_rrs510,sat_rrs555,sat_rrs670,sat_lt412,sat_lt443,sat_lt490,sat_lt510,...,sat_rhot510,sat_rhot555,sat_rhot670,sat_file,sat_rho_rc412,sat_rho_rc443,sat_rho_rc490,sat_rho_rc510,sat_rho_rc555,sat_rho_rc670
47,0.00271,0.00336,0.00344,0.00257,0.00148,0.00005,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,S1998270210316.L2_MLAC,0.004873,0.005417,0.005583,0.004854,0.003908,0.002249
259,0.00312,0.00272,0.00315,0.00280,0.00189,0.00033,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,S1999029125458.L2_MLAC,0.004175,0.004056,0.004442,0.004098,0.003330,0.001712
431,0.00062,0.00101,0.00168,0.00222,0.00369,0.00105,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,S2007125172249.L2_MLAC,0.008051,0.008214,0.008475,0.008718,0.009881,0.006990
432,0.00083,0.00118,0.00171,0.00211,0.00355,0.00131,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,S2007126180310.L2_MLAC,0.004872,0.005095,0.005414,0.005687,0.006682,0.004150
433,0.00190,0.00252,0.00356,0.00368,0.00370,0.00048,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,S2007128174513.L2_MLAC,0.003303,0.003848,0.004685,0.004880,0.004931,0.001915


In [18]:
df_2.filter(regex='sat').loc[df_2.sat_rhot412==0]

,sat_rrs412,sat_rrs443,sat_rrs490,sat_rrs510,sat_rrs555,sat_rrs670,sat_lt412,sat_lt443,sat_lt490,sat_lt510,...,sat_rhot510,sat_rhot555,sat_rhot670,sat_file,sat_rho_rc412,sat_rho_rc443,sat_rho_rc490,sat_rho_rc510,sat_rho_rc555,sat_rho_rc670
47,0.00271,0.00336,0.00344,0.00257,0.00148,0.00005,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,S1998270210316.L2_MLAC,0.004873,0.005417,0.005583,0.004854,0.003908,0.002249
259,0.00312,0.00272,0.00315,0.00280,0.00189,0.00033,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,S1999029125458.L2_MLAC,0.004175,0.004056,0.004442,0.004098,0.003330,0.001712
431,0.00062,0.00101,0.00168,0.00222,0.00369,0.00105,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,S2007125172249.L2_MLAC,0.008051,0.008214,0.008475,0.008718,0.009881,0.006990
432,0.00083,0.00118,0.00171,0.00211,0.00355,0.00131,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,S2007126180310.L2_MLAC,0.004872,0.005095,0.005414,0.005687,0.006682,0.004150
433,0.00190,0.00252,0.00356,0.00368,0.00370,0.00048,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,S2007128174513.L2_MLAC,0.003303,0.003848,0.004685,0.004880,0.004931,0.001915


In [15]:
df_2.filter(regex='sat').loc[df_2.sat_rho_rc412==0]

,sat_rrs412,sat_rrs443,sat_rrs490,sat_rrs510,sat_rrs555,sat_rrs670,sat_lt412,sat_lt443,sat_lt490,sat_lt510,...,sat_rhot510,sat_rhot555,sat_rhot670,sat_file,sat_rho_rc412,sat_rho_rc443,sat_rho_rc490,sat_rho_rc510,sat_rho_rc555,sat_rho_rc670
174,0.01201,0.00999,0.00758,0.00524,0.00307,0.0006,11.50068,10.42751,8.11119,6.78725,...,0.12322,0.09268,0.05787,S2001240175857.L2_MLAC,0.0,0.0,0.0,0.0,0.0,0.0


#### <u>Subsetting data</u>

In [17]:
swf_bands = [412, 443, 490, 510, 555, 670]
time_loc_cols_extract = ['datetime', 'lat', 'lon']
anc_cols_extract = ['oisst', 'wt', 'sal', 'etopo2', 'sola', 'solz']
sat_cols_extract = ['sat_rrs%d' %b for b in swf_bands]\
                     + ['sat_lt%d' %b for b in swf_bands]\
                     + ['sat_rhot%d' %b for b in swf_bands]\
                     + ['sat_rho_rc%d' %b for b in swf_bands]

In [18]:
ad_cols_extract = df_2.filter(regex='ad[0-9]+').columns.tolist()
ag_cols_extract = df_2.filter(regex='ag[0-9]+').columns.tolist()
ap_cols_extract = df_2.filter(regex='ap[0-9]+').columns.tolist()
bb_cols_extract = df_2.filter(regex='bb[0-9]+').columns.tolist()
chl_cols_extract = ['chl', 'chl_a']

In [19]:
df_anc = df_2[time_loc_cols_extract + anc_cols_extract]
df_sat = df_2[time_loc_cols_extract + sat_cols_extract]
df_ad = df_2[time_loc_cols_extract + ad_cols_extract]
df_ag = df_2[time_loc_cols_extract + ag_cols_extract]
df_ap = df_2[time_loc_cols_extract + ap_cols_extract]
df_bb = df_2[time_loc_cols_extract + bb_cols_extract]
df_chl = df_2[time_loc_cols_extract + chl_cols_extract]

In [20]:
pkldir = './pickleJar/'
df_anc.to_pickle(pkldir + 'df_2_ancillary.pkl')
df_sat.to_pickle(pkldir + 'df_2_satellite.pkl')
df_ad.to_pickle(pkldir + 'df_2_ad.pkl')
df_ag.to_pickle(pkldir + 'df_2_ag.pkl')
df_ap.to_pickle(pkldir + 'df_2_ap.pkl')
df_bb.to_pickle(pkldir + 'df_2_bb.pkl')

#### <u>Creating distinction between hplc and fluo chl</u>

In [21]:
df_chl.replace(to_replace=-999, value=np.NaN, inplace=True)

In [22]:
df_chl.rename(columns={'chl': 'chl_fluo', 'chl_a': 'chl_hplc'}, inplace=True)

In [66]:
def get_chl(row):
    if pd.isna(row.chl_hplc):
        return row.chl_fluo
    return row.chl_hplc

df_chl['chl'] = df_chl.apply(get_chl, axis=1)

In [68]:
df_chl['is_hplc'] = ~df_chl.chl_hplc.isnull()

df_chl.to_pickle(pkldir + 'df_2_chl.pkl')

#### <u> Creating phytoplankton absorption DataFrame</u>

In [25]:
a_bb_bands = list(df_ap.columns.str.extract('([0-9]+)').dropna().values.squeeze())

In [26]:
df_aphy = pd.DataFrame(columns=time_loc_cols_extract + ['aphy%s' % b for b in a_bb_bands])
df_aphy[time_loc_cols_extract] = df_2[time_loc_cols_extract]

In [27]:
for b in a_bb_bands:
    df_aphy['aphy%s' %b] = df_ap['ap%s' % b] - df_ad['ad%s' %b]

In [79]:
df_aphy.head().T

,0,1,2,3,4
datetime,1997-10-11 09:32:00,2000-02-22 17:00:00,2001-02-19 16:10:00,2002-01-22 13:45:00,1997-09-27 11:29:00
lat,39.29,-61.45,-61.29,-60.999,24.1392
lon,25.11,-62.299,-56.29,-56.498,-20.9995
aphy405,0,0,0.02149,0.01693,0
aphy411,0,0,0.0241,0.01886,0
aphy443,0,0,0.03078,0.02283,0
aphy455,0,0,0.02838,0.02071,0
aphy465,0,0,0.02765,0.01999,0
aphy489,0,0,0.02057,0.01464,0
aphy510,0,0,0.01261,0.00937,0


In [29]:
df_aphy.to_pickle(pkldir + 'df_2_aphy.pkl')

#### <u>Creating Dataset for OO Conference 2018</u>

I will create two dataset. Both will include time, location, and ancillary data. The first dataset will include chlorophyll as target; the second dataset will include phytoplankton absorption (aphy) as target. 

In [69]:
# Target: chl
df_ml_chl = pd.concat((df_anc.drop(['datetime', 'lat', 'lon'], axis=1),
                       df_sat.filter(regex='sat_rho_rc'), df_chl), axis=1)
df_ml_chl.replace(to_replace=-999, value=np.NaN, inplace=True)
df_ml_chl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 19 columns):
oisst            495 non-null float64
wt               158 non-null float64
sal              96 non-null float64
etopo2           495 non-null float64
sola             495 non-null float64
solz             495 non-null float64
sat_rho_rc412    495 non-null float64
sat_rho_rc443    495 non-null float64
sat_rho_rc490    495 non-null float64
sat_rho_rc510    495 non-null float64
sat_rho_rc555    495 non-null float64
sat_rho_rc670    495 non-null float64
datetime         495 non-null datetime64[ns]
lat              495 non-null float64
lon              495 non-null float64
chl_fluo         364 non-null float64
chl_hplc         158 non-null float64
chl              424 non-null float64
is_hplc          495 non-null bool
dtypes: bool(1), datetime64[ns](1), float64(17)
memory usage: 74.0 KB


In [38]:
# dropping wt and sal for too sparse data content
df_ml_chl.drop(['wt', 'sal'], axis=1, inplace=True)

In [80]:
# Target: aphy
df_ml_aphy = pd.concat((df_anc.drop(['datetime', 'lat', 'lon'], axis=1)
                        , df_sat.filter(regex='sat_rho_rc'), df_aphy), axis=1)
df_ml_aphy.drop(['wt', 'sal'], axis=1, inplace=True)

In [81]:
df_ml_aphy.describe().T

,count,mean,std,min,25%,50%,75%,max
oisst,495.0,19.066020,6.983298,0.780,13.775000,19.600000,25.120000,30.510000
etopo2,495.0,768.961616,1410.753061,0.000,20.500000,156.000000,519.000000,5529.000000
sola,495.0,188.826667,51.128164,0.000,182.750000,191.900000,204.100000,356.700000
solz,495.0,31.291919,14.525706,0.000,20.650000,32.300000,42.600000,69.900000
sat_rho_rc412,495.0,0.006742,0.003876,0.000,0.003706,0.005988,0.008865,0.020979
sat_rho_rc443,495.0,0.007025,0.003640,0.000,0.004168,0.006625,0.008941,0.020130
sat_rho_rc490,495.0,0.007243,0.003453,0.000,0.004514,0.006866,0.009144,0.018906
sat_rho_rc510,495.0,0.006734,0.003350,0.000,0.004123,0.005819,0.008748,0.018512
sat_rho_rc555,495.0,0.006170,0.003526,0.000,0.003430,0.005138,0.007997,0.019804
sat_rho_rc670,495.0,0.003537,0.002458,0.000,0.001666,0.002724,0.004723,0.011320


In [50]:
df_ml_aphy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 33 columns):
oisst            495 non-null float64
etopo2           495 non-null float64
sola             495 non-null float64
solz             495 non-null float64
sat_rho_rc412    495 non-null float64
sat_rho_rc443    495 non-null float64
sat_rho_rc490    495 non-null float64
sat_rho_rc510    495 non-null float64
sat_rho_rc555    495 non-null float64
sat_rho_rc670    495 non-null float64
datetime         495 non-null datetime64[ns]
lat              495 non-null float64
lon              495 non-null float64
aphy405          495 non-null float64
aphy411          495 non-null float64
aphy443          495 non-null float64
aphy455          495 non-null float64
aphy465          495 non-null float64
aphy489          495 non-null float64
aphy510          495 non-null float64
aphy520          495 non-null float64
aphy530          495 non-null float64
aphy550          495 non-null float64
aphy555       

In [82]:
df_ml_chl.to_pickle(pkldir + 'df_3_ML4chl.pkl')
df_ml_aphy.to_pickle(pkldir + 'df_3_ML4aphy.pkl')